In [1]:
import pandas as pd

In [2]:
data3_path=r"C:\Users\gauta\OneDrive\Desktop\Untitled Folder\MapUp-Data-Assessment-F-main\datasets\dataset-3.csv"

In [23]:
data2=pd.read_csv(r"C:\Users\gauta\OneDrive\Desktop\Untitled Folder\MapUp-Data-Assessment-F-main\datasets\dataset-3.csv")

In [3]:
data3=pd.read_csv(data3_path)
data3.head()

,id_start,id_end,distance
0,1001400,1001402,9.7
1,1001402,1001404,20.2
2,1001404,1001406,16.0
3,1001406,1001408,21.7
4,1001408,1001410,11.1


In [8]:
def calculate_distance_matrix(df)->pd.DataFrame:
    #Create a pivot table to represent distances between toll locations
    pivot_df = df.pivot(index='id_start', columns='id_end', values='distance').fillna(0)

    #Convert pivot table to DataFrame and ensure symmetry by adding transposed matrix
    df = pivot_df.add(pivot_df.T, fill_value=0)

    # Set diagonal values to 0
    for col in df.columns:
        df.loc[col, col] = 0

    return df


In [9]:
distance_matrix = calculate_distance_matrix(data3)
print(distance_matrix)

         1001400  1001402  1001404  1001406  1001408  1001410  1001412  \
1001400      0.0      9.7      0.0      0.0      0.0      0.0      0.0   
1001402      9.7      0.0     20.2      0.0      0.0      0.0      0.0   
1001404      0.0     20.2      0.0     16.0      0.0      0.0      0.0   
1001406      0.0      0.0     16.0      0.0     21.7      0.0      0.0   
1001408      0.0      0.0      0.0     21.7      0.0     11.1      0.0   
1001410      0.0      0.0      0.0      0.0     11.1      0.0     15.6   
1001412      0.0      0.0      0.0      0.0      0.0     15.6      0.0   
1001414      0.0      0.0      0.0      0.0      0.0      0.0     18.2   
1001416      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001418      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001420      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001422      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001424      0.0      0.0      0.0    

In [10]:
def unroll_distance_matrix(distance_matrix)->pd.DataFrame:
    #Reset index to get 'id_start' as a column
    distance_matrix = distance_matrix.reset_index()

    #Melt the DataFrame to transform it to long format
    melted_df = pd.melt(distance_matrix, id_vars='index', var_name='id_end', value_name='distance')

    #Rename columns for clarity
    melted_df.columns = ['id_start', 'id_end', 'distance']

    #Filter out same id_start and id_end pairs
    df = melted_df[melted_df['id_start'] != melted_df['id_end']].reset_index(drop=True)

    return df


In [11]:
unrolled_df = unroll_distance_matrix(data3)
print(unrolled_df)

     id_start    id_end   distance
0           0  id_start  1001400.0
1           1  id_start  1001402.0
2           2  id_start  1001404.0
3           3  id_start  1001406.0
4           4  id_start  1001408.0
..        ...       ...        ...
127        39  distance       26.7
128        40  distance        8.5
129        41  distance       10.7
130        42  distance       10.6
131        43  distance       16.0

[132 rows x 3 columns]


C:\Users\gauta\AppData\Local\Temp\ipykernel_13916\1466578341.py:6: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  melted_df = pd.melt(distance_matrix, id_vars='index', var_name='id_end', value_name='distance')


In [24]:
def find_ids_within_ten_percentage_threshold(df, reference_value)->pd.DataFrame:
    #Calculate the average distance for the reference value
    avg_distance_reference = df[df['id_start'] == reference_value]['distance'].mean()

    #Calculate the threshold range (10% of the average distance)
    threshold = 0.1 * avg_distance_reference

    #Filter 'id_start' values within the threshold range
    within_threshold_ids = df[(df['id_start'] != reference_value) &
                              (df['distance'] >= avg_distance_reference - threshold) &
                              (df['distance'] <= avg_distance_reference + threshold)]['id_start'].unique()

    #Sort the 'id_start' values within the threshold
    value = sorted(within_threshold_ids)

    return value


In [28]:
result_ids_within_threshold = find_ids_within_ten_percentage_threshold(unrolled_df, 10)
print(result_ids_within_threshold)

[]


In [16]:
def calculate_toll_rate(df):
    #Calculate toll rates based on distance and rate coefficients for each vehicle type
    df['moto'] = df['distance'] * 0.8
    df['car'] = df['distance'] * 1.2
    df['rv'] = df['distance'] * 1.5
    df['bus'] = df['distance'] * 2.2
    df['truck'] = df['distance'] * 3.6

    return df


In [27]:
result_with_toll_rates = calculate_toll_rate(unrolled_df)
print(result_with_toll_rates)

     id_start    id_end   distance       moto         car          rv  \
0           0  id_start  1001400.0  801120.00  1201680.00  1502100.00   
1           1  id_start  1001402.0  801121.60  1201682.40  1502103.00   
2           2  id_start  1001404.0  801123.20  1201684.80  1502106.00   
3           3  id_start  1001406.0  801124.80  1201687.20  1502109.00   
4           4  id_start  1001408.0  801126.40  1201689.60  1502112.00   
..        ...       ...        ...        ...         ...         ...   
127        39  distance       26.7      21.36       32.04       40.05   
128        40  distance        8.5       6.80       10.20       12.75   
129        41  distance       10.7       8.56       12.84       16.05   
130        42  distance       10.6       8.48       12.72       15.90   
131        43  distance       16.0      12.80       19.20       24.00   

            bus       truck  
0    2203080.00  3605040.00  
1    2203084.40  3605047.20  
2    2203088.80  3605054.40  
3  

In [32]:
import datetime
def calculate_time_based_toll_rates(df):
    # Convert start and end timestamps to datetime format
    df['start_time'] = pd.to_datetime(df['start_time'])
    df['end_time'] = pd.to_datetime(df['end_time'])

    # Create time range objects for different time intervals
    weekdays_morning = (datetime.time(0, 0), datetime.time(10, 0))
    weekdays_afternoon = (datetime.time(10, 0), datetime.time(18, 0))
    weekdays_evening = (datetime.time(18, 0), datetime.time(23, 59, 59))
    weekends_all_day = (datetime.time(0, 0), datetime.time(23, 59, 59))

    # Define functions to apply discount factors based on time intervals
    def apply_discount(row):
        if row['start_time'].weekday() < 5:  # Weekdays (Monday to Friday)
            if weekdays_morning[0] <= row['start_time'].time() <= weekdays_morning[1]:
                return row * 0.8
            elif weekdays_afternoon[0] <= row['start_time'].time() <= weekdays_afternoon[1]:
                return row * 1.2
            elif weekdays_evening[0] <= row['start_time'].time() <= weekdays_evening[1]:
                return row * 0.8
        else:  # Weekends (Saturday and Sunday)
            return row * 0.7

    # Apply discount factors to vehicle columns based on time intervals
    vehicle_columns = ['moto', 'car', 'rv', 'bus', 'truck']
    for col in vehicle_columns:
        df[col] = df[col].apply(apply_discount)

    # Add columns for start_day and end_day
    df['start_day'] = df['start_time'].dt.day_name()
    df['end_day'] = df['end_time'].dt.day_name()

    return df


In [33]:
# Assuming 'time_based_toll_df' is the DataFrame from Question 3
result_with_time_based_toll_rates = calculate_time_based_toll_rates(result_ids_within_threshold)
print(result_with_time_based_toll_rates)

TypeError: list indices must be integers or slices, not str